## Part 1 - Introduction

In this exercise, we will be analyzing RNA-Seq data from Liu et al. (Genomics Data 7 (2016) 144–147. Transcriptional profiling of the epigenetic regulator Smchd1). They created Smchd1 knock-out mice and studied the changes in expression in neural stem cells. Smchd1 is an epigenetic repressor that plays a role in, amongst others, X-inactivation.

The paper can be found on the home page.


#### Preparation
The following only needs to be done only once (and may in fact already
be installed). That's why the code (`source(...)`) is turned into a
comment, using `#`

In [18]:
# Get the biocLite installer script ...
# source("http://www.bioconductor.org/biocLite.R")

In [19]:
# ... and use it to install the DESeq2 package (again, this may already be installed)
# biocLite("DESeq2")

Now load this package into your current workspace. 

Documentation can be obtained by putting a question mark in front of the thing
you need help on:

In [20]:
?library

If DESeq2 is (was already) installed, you can now load it into the current workspace:


In [21]:
library(DESeq2)

DESeq2 is an extensive and well-regarded package to analyze RNA-Seq data, especially differential expression ('DE') between genes between different conditions, typically wild-type and some condition, in this case the knock-down of gene `Smchd1`.


The documentation often contains an example section. This is useful in itself, but the example that is given can actually be run using the example() function. For instance, in a moment we will load some data using the `load()` function, you can run its example code as follows.


In [22]:
example(load)

Let's load the 'Mouse Neural Stem Cell data from Liu et al.' (GSE65747.rda)
This data has already been prepared by us. It contains just one big data 
object representing a complete data set.

In [23]:
load('GSE65747.rda')

What is the name of the object you just loaded (use the `ls()` function)?


In [64]:
ls()

What is the type of this object? (use the `class()` or the `is()` function)

In [25]:
class(dds)
is(dds)

The object is a 'DESeqDataSet', which is essentially a big table with
the read counts per gene and per sample, but also contains the
'metadata', i.e.  the details of the genes (rows) and samples
(columns).  You can get a quick overview of the data by just typing
in the name of the object in the R console. How many genes and how
how many samples are contained in the object?


In [26]:
dds

Looking up (meta)data inside the dds object is done using the
functions `counts()` for the actual counts, function `colData()` for
information per column (such as which experimental group a sample
belongs to), and `mcols()` or its synonym `values()` for the metadata on genes.
What metadata is there per sample?


In [27]:
colData(dds)

What metadata is available for each gene?

In [28]:
mcols(dds)

Have a look at the actual counts. Use the `head()` function to limit the
output, and `summary()` for a summary.

In [29]:
head(counts(dds))                       # head() gives the first rows
summary(counts(dds))                    # summary statistics per column

Which sample had most, and which had fewest mapped reads?

You can look up the exact counts by specifying the gene (row) and
sample (column) in matrix notation, using square brackets and a comma.
E.g. look up gene number 1000, in
sample 3:


In [30]:
counts(dds[1000,3])

To specify a numeric range, use the 'colon operator'. E.g., to show
the counts for gene number 101 to number 105' for samples 2 and 3, do:

In [31]:
counts(dds[101:105, 2:3])

But it's clearer to select rows and colums by rowname and/or column name (if
they are present; not all matrices have row and/or column names)

In [32]:
counts(dds['Smap1', 'A'])

The thing before the comma is called the _row index_ and it selects the rows.
The thing after the comma is the _column index_ and selects the columns.
If you leave an index unspecified, it will select _everything_ of that dimension:

Is gene 'Malat1' abudant or not?

For data objects of type `DESeqDataSet` you can also use this row and/or column selection mechanism to lookup the corresponding metadata using the `mcols` function.  What, for instance, is the metadata that the object has on the Rp1 gene?


In [33]:
mcols(dds['Rp1',])

What information is given per gene? On which chromosome does gene 'Vcp' lie?

What are the maximum and mininum counts for each of the samples? 
Use the `min()`, `max()` and/or `summary()` functions, e.g.

In [34]:
counts(dds['Hdhd2',    ]) ## note the 'comma-nothing'!

In [35]:
max(counts(dds[, 'B']))

Likewise, you can obtain the usual statistical functions using
`mean(), median(), var(), sd(), sum()` etc. Try this

You can 'save' values in variables using assignment, which looks like this

In [36]:
Amean <- mean(counts(dds[,'A']))

What are the counts for the Smchd1 gene in the wild-type samples,
and what are they in the Smchd1-null samples? Why? 

To get an impression of the distribution of read counts of a sample, simply pass it to the `hist()` function.

In [37]:
hist(counts(dds[,'C']))

To see more details you will have to specify a larger number of
histogram bins by specifying an additional `nclass=SOMENUMBER` argument
to the `hist()` function. This looks like

In [38]:
hist(counts(dds[,'C']),  nclass=100)

To 'zoom in', e.g. to just show the bit between 0 and twice the
mean of the the read counts (you just saved that in a variable,
right?), you have to specify another optional argument, namely
`xlim=c(lower, upper)`. Try this (and adjust the `nclass` argument if
needed).

Now let's compare the distributions of _all_ the data sets. An easy way
to do that is the `boxplot()` function.  When given a table such as
returned by `counts(dds)` it will do a boxplot for each of the columns
in the matrix.

In [39]:
boxplot(counts(dds))

Supply an optional ylim-argument (such as `... ylim=c(0, 1000) ...` )
to make the differences between
the samples clearer. Which of the
samples is the 'odd one out', based on this? Is it a knock-out or wt
genotype?

#### End of part 1 ####

## Part 2 - Statistics

The theory behind the differential expression analysis is
fairly complex. For one thing, the counts samples have to be 
normalized: i.e., made more comparable by e.g. 
correcting for read depth. This normalization as well as the statistical
evaluation is done in one call to DESeq:

In [40]:
dds <- DESeq(dds)                       #note: this adds things to the 'dds' object

Amongst others, the data has now been normalized. This is visible using 
the `colData()`
What is the normalization factor for the 'odd-one out' sample
from the previous exercise?

In [41]:
colData(dds)

To get the read counts after normalization, specify
`normalized=TRUE` as an extra argument to `counts()`. Compare the
boxplots of the unnormalized data (done in the last exercise of the
previous session) with those of normalized data. Did the
normalization work?

In [42]:
boxplot(counts(dds, normalized=TRUE), ylim=c(0,2000))

To get the statistical results out of the normalized data,
use the `results()` function. It needs the `DESeqDataSet` and
a so-called _contrast_. A contrast specifies what experimental factor to
compare (here: 'group'), which samples are 'treatment', and
which samples are 'control'. It returns a table-like data
structure. Let's assign it to a new variable called `res` :

In [43]:
res <- results(dds, contrast=c("group", "Smchd1-null", "WT"))
res

The `summary()` function again gives a useful overview of the results
How many outliers are there, and how many 'low counts'?

In [44]:
summary(res)

To get an impression of the data as a whole, the change per
gene versus its average is plotted. Use the `plotMA()` function for this,
and give it the `res` object as an argument.

In [45]:
plotMA(res)

By default, `plotMA()` tries to show most of the data, and chooses
its own y-axis limits. Genes outside the range are shown as
triangles. Play with the `ylim` argument to show all the data. Better
yet, use `min()` and `max()` on the 'log2FoldChange' column of your
`results` data to find the limits automatically. To make the `min()` and
`max()` functions ignore the NA's (i.e. the missing data), you have to also 
pass an `na.rm=TRUE` argument.

In [46]:
lowest <- min(res[,'log2FoldChange'], na.rm=TRUE)
highest <- max(res[,'log2FoldChange'], na.rm=TRUE)
plotMA(res, ylim=c(lowest,highest))

Have a look at e.g. the first 10 rows of the `results` table.  What
do the columns mean? Why is `padj` greater than `pvalue`?  What are the
statistics for the Smchd1 gene? (Remember how you selected data on a
particular gene in one of the previous exercises).

In [47]:
res[1:10, ]
res['Smchd1', ]

The genes Ndn, Mkrn3 and Peg12 are known to be repressed by
Smchd1. Do the statistics confirm this?

In [48]:
res['Ndn',]
res['Mkrn3',]
res['Peg12',]

To find the top 10 genes that, in the Smchd1 knock-out, go down or go
up most, we have first have to sort the results table. In R, this is
done as follows:

In [49]:
order.incr <- order(res[, 'log2FoldChange'])
res.incr <- res[order.incr, ]

order.decr <- order(res[, 'log2FoldChange'], decreasing=TRUE)
res.decr <- res[order.decr, ]

The `order()` function simply calculates a vector of numbers that puts the rows of
the table in the the right order. By default, the ordering is from
low to high; to get a descending order, specify `decreasing=TRUE` as
an extra argument to `order()` 

Now find the 10 genes that go up most, and those that go down most

#### End of Part 2 ####

## Part 3 - Gene Selections

In order to do further analyses such as gene set enrichment, we need
to make selections of our genes, based on the fold-change and
adjusted p-value (adjusted for _what_ ?). You can use the `order()` function, but sometimes you
want to specify an explicit threshold instead of 'the top so-many'. For
this we need comparison operators. E.g. to find all the genes without
any counts, we could say:

In [50]:
not.expressed <- (res[,'baseMean'] == 0)   # NOTE: the double equals-sign == means 'is equal to',  single '=' is used for function arguments!

`not.expressed` is now a vector of so-called logicals (als known as booleans),
with `TRUE` where the `baseMean`- column had a 0, and `FALSE` where not.

You can use this logical vector to select rows from the original table:

In [51]:
res[not.expressed, ]

Note the 'comma-nothing' again (we are selecting specific rows using the `not.expressed` vector, but for these rows, we select *all* their columns)


This vector of TRUE and FALSE is all very nice, but we want to find out how many were not expressed, not which ones. This is a very common task, and luckily there is a function to help us: sum() gives you the number of TRUEs in a boolean vector. (The reason is that all the FALSEs and TRUEs are first automatically converted to 0's and 1's respectively)

In [52]:
sum(not.expressed)

The genes without any counts lead to NA's, that is, missing values in the results table (`NA` stands for 'not available'). We have to get rid of them because these values are 'contageous': many functions will return NA if they are given arguments containing any NA. Functions such as `mean()`, `min()` and `max()` all have a special optional argument `na.rm` to remove the NA's before doing the actual calculation.  However, those functions only work on vectors, not on matrices and tables so we need something more general: a function that returns `TRUE` for NA, and `FALSE` otherwise. That function is called `is.na()`:

In [53]:
na <- is.na(res[   , 'padj']) # note the 'nothing - comma' here

How many of these `NA` values are there?


We now want use our 'na' vector to select things from the results table, but we need in fact the exact
opposite: all the rows that are *not* `NA`.  To negate logical values
there is the `!`  operator (also called the NOT operator)

In [54]:
available <- !na

How many results do have an adjusted p-value available?

We can now create a set with only the clear results:

In [55]:
res2 <- res[available, ]

Which genes have a `padj` value better than 1e-20?

In [56]:
sum( res2[   , 'padj']  < 1e-20 ) # again, note the 'nothing - comma' here

Lastly, we should be able to combine several comparison operators,
e.g. select genes that *both* have changed at least 2-fold up, AND also have a
p-value better than 0.01. For this there is the AND operator: `&` .  It
works as can be expected: `a & b` is only `TRUE` if both `a` AND `b` have
`TRUE` (note: this is done along the whole length of the vector of `TRUE`s and `FALSE`s)

How many genes go up, and have an adjusted p-value better than
0.01, and how many satisfy both criteria?

In [57]:
sign_up <-  res2[ , 'padj'] < 0.01 & res2[ , 'log2FoldChange'] > 1
sum(sign_up)

Same question for the genes going down significantly at least a factor 2: 

In [58]:
sign_down <-  res2[ , 'padj'] < 0.01 & res2[ , 'log2FoldChange'] < -1 # space between '<' and '-' (otherwise it's an assignment!)
sum(sign_down)

We can now try to see what is so special about this set of genes. 
You just created a vector of booleans that you can use to
select them from the original table. What you need are their
names, which you can extract using the `rownames()` function:

In [59]:
signup_genes <- rownames(res2[sign_up,  ])

To get these gene names nicely formatted for easy copy-pasting,
use the `cat()` function with `"\n"` (meaning: new line) as the separator:

In [60]:
cat(signup_genes, sep="\n")

Use http://www.geneontology.org/page/go-enrichment-analysis ) to
analyse your list of gene names (be sure to select Mouse as
species). Which processes, functions and/or components are
overrepresented in your up list? Which group of genes is
responsible for the most significant terms in the up-list?

Note: if the geneontology site is down, use
http://integromics.holstegelab.nl/index.php?framesrc=Core/go.cgi&action=search
(the password will be on the whiteboard)

#### End of part 3 ####

## Glimma demo

This exercise is an extra, to give you a taste of the capabilities of R. 
R can produce rich interactive documents. An important example of this is
the `Shiny` library. Here we are using `Glimma` library, which is a bit
easier, and especially suited to display expression data. The Glimma package
may already be installed; if not we'll have to do it ourselves, but first try:

In [76]:
library(Glimma)

If that did't work we'll install it ourselves, but we are not allowed to install things to the default location, so we'll first set up our own location and install it there. It may also decide to install the `edgeR` package, and it may complain (which you can ignore) about not being able to update the `MASS` package.

In [86]:
# lib.dir <- paste(sep="/", Sys.getenv("HOME"), 'Rlibs')
# dir.create(lib.dir)
# source("http://www.bioconductor.org/biocLite.R")
# biocLite("Glimma", lib=lib.dir)
### alternatively: 
# library(devtools)
# devtools::install_github("Shians/Glimma")

Try to load it again, this time also specify where to load the library from:

In [81]:
library(Glimma, lib.loc=lib.dir)

To highlight the genes that have a p-value better than, say, 0.01, 
we can supply a vector of zeroes (black) and ones (red), based 
on the adjusted p-value. The old version of Glimma cannot deal with `NA`'s,
so we have to manually hide them by giving them a proper (but non-significant!)
p-value, say, 1

In [82]:
padj <- res[, 'padj']
padj[ is.na(padj) ] <- 1

Genes that will be displayed as red dots must have a `status` of one, so they must have a `padj` smaller than
0.01. This gives a boolean vector, but unfortunately, Glimma only
accepts zeroes and ones. We can convert `FALSE` and `TRUE` to 0 and 1 using
the `as.numeric()` function

In [83]:
status <- as.numeric(padj < 0.01)

A few more things are needed:

In [84]:
groups <- colData(dds)[ ,'group'] # which experimental factors to take along
colors <- c('blue', 'LightBlue', 'red', 'pink', 'DarkRed', 'DarkBlue') # what color to give them
display <- c("GeneID", "GeneName", "logFC") # which annotation (from mcols(dds) ) and numeric data to show

Now we are ready to produce the interactive HTML document. Running this function can take up to a minute (wait until the green dot disappears). 

In [85]:
glMDPlot(res,
         status=status,
         counts=counts(dds),
         samples=colnames(dds),
         sample.cols=colors,
         anno=mcols(dds),
         groups=groups,
         display.columns=display)

After this, you to the `Files` menu (top-left corner).
There, select the `glimma-dir` folder (by ticking it, not clicking) it.
Then, click the `Compress` button (you can specify your own name if you like).
This prepares a ZIP file that you should download (tick the box and click `Download`)
If you open this file on your own computer, you should navigate tot he `MD-Plot.html` 
Open it by double-clicking and enjoy the interactivity. Which genes have the highest expression, in general?


It may be nicer to display the expression levels in the right-handside plot on a log scale. To this end, redo the whole thing while supplying a `side.log` argument. This also requires that there are no zeroes in the data; you can solve this by supplying a `counts = counts(dds)+1 ` argument.